### **Actualización 1.5:**

---


*   Plot animado
*   Introducir masas en FNBody






# Import Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print('Enter the path:')
path = input()
import sys
sys.path.insert(0,'/content/drive/MyDrive/'+path)


# Upload modules manually

In [ ]:
from google.colab import files
files.upload()

# Import functions

In [ ]:
import numpy as np
import math
from numba import cuda
from scipy.optimize import newton
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

from Cauchy_Problem import Cauchy_problem
from Temporal_Schemes import Euler, Crank_Nicolson, RK4, Inverse_Euler, LeapFrog
from fNbody import F_NBody
from initialConditions import initial_cond
from Physics import Kepler

# Información de la GPU

In [ ]:
from numba import cuda
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("maxThreadsPerMultiprocessor = %s" % str(gpu.MAX_THREADS_PER_MULTI_PROCESSOR))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))

cuda.detect()

# main

In [ ]:
print('Enter number of bodies:')
Nb = int(input()) # 1024*72*2 # nº cuerpos tiene que ser 1024 (nº threads por bloque) * 72 (nº bloques o SM) * A (nº entero a elegir)
Nc = 3 # nº coord

t0 = 0 # timepo inicial
print('Enter time [s]:')
tf = int(input()) # tiempo final
N = 10*tf # number of time steps
t = np.linspace(t0, tf, N)

U0, M = initial_cond(Nb,Nc)

U = Cauchy_problem(RK4, F_NBody, t, U0, Nb, M)

U_s = np.reshape( U, (N, Nb, 2, Nc) ) 
r = np.reshape( U_s[:, :, 0, :], (N, Nb, Nc) )
print('Done!')





# 3D plot animation

In [ ]:
# ANIMATION FUNCTION
def func(num, dataSet, line):
    # NOTE: there is no .set_data() for 3 dim data...
    line.set_data(dataSet[0:2, :num])    
    line.set_3d_properties(dataSet[2, :num])    
    return line
 

# THE DATA POINTS
x = r[:, 0, 0]
y = r[:, 0, 1]
z = r[:, 0, 2] 
dataSet = np.array([x, y, z])
numDataPoints = len(x)
 
# GET SOME MATPLOTLIB OBJECTS
fig = plt.figure()
ax = Axes3D(fig)
fig.set_size_inches(w_in_inches, h_in_inches, True)
 
# NOTE: Can't pass empty arrays into 3d version of plot()
line = plt.plot(dataSet[0], dataSet[1], dataSet[2], lw=2, c='g')[0] # For line plot
 
# AXES PROPERTIES]
# ax.set_xlim3d([limit0, limit1])
ax.set_xlabel('X(t)')
ax.set_ylabel('Y(t)')
ax.set_zlabel('Z(t)')
ax.set_title(f'N = {Nb} body problem: 3D projection')
 
# Creating the Animation object
line_ani = animation.FuncAnimation(fig, func, frames=numDataPoints, fargs=(dataSet,line), interval=50, repeat=False, blit=False)
f = r"/content/drive/MyDrive/" + path + "/animation.mp4" 
writervideo = animation.FFMpegWriter(fps=60) 
line_ani.save(f, writer=writervideo, dpi=300)
 
 
#plt.show()
